<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/CodeBERT_1_N_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [4]:
import pandas as pd

In [5]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/final_train5.csv')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df = df[['CWE ID','CODE', 'IDX']]

In [ ]:
df.columns

Index(['CODE', 'IDX'], dtype='object')

In [ ]:
df = df[df['CWE ID'] != 'CWE-119']

In [6]:
df = df[['CWE ID','code', 'labels']]

In [6]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
batch_size = 20
epoch_num = 1000
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [64]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    preprocess_df = df.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df = preprocess_df.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df = preprocess_df.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df = preprocess_df.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df = df.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df.to_csv(file_name, index=False)

def tokenized(batch):
    # 결과를 저장할 딕셔너리를 초기화합니다.
    tokenized_batch = {}
    for column in batch:
        # 각 컬럼 데이터를 문자열 리스트로 변환합니다. None 값 또는 다른 타입이 있을 경우 문자열로 처리합니다.
        texts = [str(text) if text is not None else "" for text in batch[column]]
        # 토큰화 실행
        tokenized_output = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_token_type_ids=True)
        # 토큰화 결과에서 필요한 부분만 추출하여 저장합니다.
        tokenized_batch[f'{column}_input_ids'] = tokenized_output['input_ids']
        tokenized_batch[f'{column}_attention_mask'] = tokenized_output['attention_mask']
    return tokenized_batch


In [65]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['code'],batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/13415 [00:00<?, ? examples/s]

In [66]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=300)

In [73]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'CWE ID', 'vul', 'lang', 'labels', 'chmod()', 'assert()', 'atoi()', 'atof()', 'calloc()', 'errno()', 'exit()', 'fclose()', 'fopen()', 'free()', 'gets()', 'read()', 'scanf()', 'printf()', 'printk()', 'write()', 'strcpy()', 'strcmp()', 'memcpy()', 'malloc()', 'kmalloc()', 'realloc()', 'puts()', 'strcat()', 'sprintf()', 'strtok()', 'signal()', 'NULL)', 'sizeof()', 'if()', 'else)', 'for()', '=)', '==)', 'while()', 'break;)', 'return 0;)', 'return 1;)', 'int*)', 'char*)', '%s)', '%c)', '%d)', '%ld)', '%x)', '%X)', '%p)', '%u)', '%lu)', '%i)', '%f)', '%lf)', '%o)', '%e)', '%g)', '%E)', '%G)', 'delete)', 'delete[])', 'class)', 'public:)', 'createArray()', 'new[])', 'memmove()', 'switch()', 'case)', 'vulnerable_strcpy', 'vulnerable_scanf', 'vulnerable_printf', 'buffer_allocation_check', 'malloc_count', 'calloc_count', 'realloc_count', 'free_count', 'malloc_status', 'calloc_status', 'realloc_status', 'File_Memory_Functions_Exis

In [67]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [68]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [69]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics


In [70]:
from transformers import TrainingArguments, Trainer
import pandas as pd
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import TrainerCallback, TrainerControl, TrainerState

class EarlyStoppingCallback(TrainerCallback):
    """ 조기 종료 콜백 구현 """
    def __init__(self, early_stopping_patience=5):
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = None
        self.patience_counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 평가 단계에서의 손실 가져오기
        eval_loss = kwargs.get('metrics', {}).get('eval_loss')
        if eval_loss is not None:
            if self.best_loss is None or eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1

            # 조기 종료 조건 체크
            if self.patience_counter >= self.early_stopping_patience:
                print(f"No improvement in {self.early_stopping_patience} evaluation steps. Stopping training early.")
                control.should_training_stop = True


# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1000,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(5)]  # EarlyStoppingCallback 추가
)

In [71]:
gc.collect()
torch.cuda.empty_cache()

In [72]:
trainer.train()

ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
trainer.save_model('model_final_train4')

In [ ]:
import pandas as pd

df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model train/final_val.csv")

In [ ]:
df2 = df2[['CODE', 'IDX']]

In [ ]:
df2

,CODE,IDX
0,nautilus_file_mark_desktop_file_trusted (GFile...,1
1,"int Track::Info::CopyStr(char* Info::*str, Inf...",1
2,long Tracks::ParseTrackEntry(long long track_s...,1
3,void HttpAuthFilterWhitelist::SetWhitelist(\n...,1
4,void CreateOAuth2ServiceDelegate(\n si...,1
...,...,...
475,static int au1200fb_ioctl(struct fb_info *info...,0
476,"static int nfs_read(struct device_d *dev, FILE...",0
477,static ChannelSecurityOptions *find_channel_se...,0
478,"uiserver_decrypt_verify (void *engine, gpgme_d...",0


In [ ]:
import re

def preprocess2(df2, file_name):
    # 멀티 라인 주석 제거
    preprocess_df2 = df2.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df2 = preprocess_df2.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df2 = preprocess_df2.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df2 = df2.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df2.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['CODE'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
preprocess2(df2,"preprocess2.csv")
dataset = load_dataset("csv",data_files="preprocess2.csv")['train']
encoded_dataset2 = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset2=encoded_dataset2.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset

Dataset({
    features: ['CODE', 'IDX'],
    num_rows: 480
})

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13414
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5750
    })
})

In [ ]:
trainer.predict(encoded_dataset['train'], metric_key_prefix="eval")

PredictionOutput(predictions=array([[-1.0813432 , -1.3311223 , -0.7123402 , ..., -0.96344745,
        -0.9476277 ,  9.178225  ],
       [-1.8125955 , -0.64799935,  0.13481271, ..., -1.09198   ,
        -1.2183334 , -1.734629  ],
       [-1.367245  ,  9.422799  , -0.4665491 , ..., -0.5583104 ,
        -0.58360183, -1.3949015 ],
       ...,
       [-0.75014985, -1.3248707 , -1.0046675 , ..., -0.35289282,
        -0.3750169 ,  9.277652  ],
       [-1.896231  ,  0.4975019 ,  8.859313  , ...,  0.02749154,
         0.6843233 , -2.2087805 ],
       [-0.81120366, -1.6978891 , -0.7074471 , ..., -1.2535216 ,
        -0.8544779 , -0.34463826]], dtype=float32), label_ids=array([10,  7,  1, ..., 10,  2,  4]), metrics={'eval_loss': 0.019206080585718155, 'eval_accuracy': 0.9937378857909647, 'eval_recall_weighted': 0.9937378857909647, 'eval_f1_weighted': 0.9937400687074712, 'eval_f1_macro': 0.9932239266629811, 'eval_f1_micro': 0.9937378857909647, 'eval_f1_class_0': 1.0, 'eval_f1_class_1': 0.9921619856

In [ ]:
import pandas as pd
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import TrainerCallback, TrainerControl, TrainerState


# 기존 검증 데이터셋 로드 및 전처리
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/final_val2.csv')
preprocess(val_df, "preprocess_val.csv")  # val_df를 전처리하고 저장

# 검증 데이터셋 로드
val_dataset = load_dataset("csv", data_files="preprocess_val.csv")['train']
val_encoded_dataset = val_dataset.map(tokenized, remove_columns=['code'], batched=True)

# 훈련 및 검증 데이터셋을 DatasetDict로 결합
full_dataset = DatasetDict({
    'train': encoded_dataset['train'],
    'test': encoded_dataset['test'],
    'validation': val_encoded_dataset
})

class EarlyStoppingCallback(TrainerCallback):
    """ 조기 종료 콜백 구현 """
    def __init__(self, early_stopping_patience=5):
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = None
        self.patience_counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 평가 단계에서의 손실 가져오기
        eval_loss = kwargs.get('metrics', {}).get('eval_loss')
        if eval_loss is not None:
            if self.best_loss is None or eval_loss < self.best_loss:
                self.best_loss = eval_loss
                self.patience_counter = 0
            else:
                self.patience_counter += 1

            # 조기 종료 조건 체크
            if self.patience_counter >= self.early_stopping_patience:
                print(f"No improvement in {self.early_stopping_patience} evaluation steps. Stopping training early.")
                control.should_training_stop = True

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='loss'
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_dataset['train'],
    eval_dataset=full_dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(5)]  # EarlyStoppingCallback 추가
)

gc.collect()
torch.cuda.empty_cache()


# 훈련 시작
trainer.train()